The objective of this notebook is to reproduce what I learn in chapter 3 with a new dataset, so we will preprocess the data and fine tune the model on a new dataset !

In [2]:
from datasets import load_dataset

raw_datasets = load_dataset("glue", "sst2")
raw_datasets


c:\Users\bapti\OneDrive\Bureau\Programmation\NLP HF COURSE\.venv\Lib\site-packages\huggingface_hub\file_download.py:140: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\bapti\.cache\huggingface\hub\datasets--glue. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Generating test split: 100%|██████████| 1821/1821 [00:00<00:00, 362704.32 examples/s]


DatasetDict({
    train: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 67349
    })
    validation: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 872
    })
    test: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 1821
    })
})

In [6]:
# Loading train dataset only
raw_train_dataset = raw_datasets["train"]
print(raw_train_dataset[2])

#checking the meaning of label
raw_train_dataset.features

{'sentence': 'that loves its characters and communicates something rather beautiful about human nature ', 'label': 1, 'idx': 2}


{'sentence': Value(dtype='string', id=None),
 'label': ClassLabel(names=['negative', 'positive'], id=None),
 'idx': Value(dtype='int32', id=None)}

In [9]:
from transformers import AutoTokenizer, DataCollatorWithPadding

checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
def tokenize_function(example):
    '''The objective of this function is to tokenize the dataset by taking in count that we have to tokenize the sentence 2 by 2 to 
    keep the pair together !'''
    return tokenizer(example["sentence"], truncation=True)   
tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

Map: 100%|██████████| 1821/1821 [00:00<00:00, 20728.83 examples/s]


In [11]:
#1 Define TrainingArguments with the dir where we'll store the finetune model
from transformers import TrainingArguments

training_args = TrainingArguments("./trainer_dir_sst2")

#2 Choose the model to finetune
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

#3 Create Trainer by passing all the arguments
from transformers import Trainer

trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
)

#4 Fine tune the model !
trainer.train()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
C:\Users\bapti\AppData\Local\Temp\ipykernel_24760\3756184751.py:14: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
c:\Users\bapti\OneDrive\Bureau\Programmation\NLP HF COURSE\.venv\Lib\site-packages\torch\nn\modules\module.py:1326: UserWarning: expandable_segments not supported on this platform (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\c10/cuda/CUDAAllocatorConfig.h:28.)
  return t.to(


Step,Training Loss
500,0.412200
1000,0.367900
1500,0.351100


KeyboardInterrupt: 